# FastAPI OpenAPI Integration Tutorial

🏦 **Learn how to connect Azure AI Agents to local FastAPI services!**

This tutorial demonstrates:
- **Starting a FastAPI service** with bank transaction data
- **Reading OpenAPI spec** from the running service
- **Creating Azure AI Agents** that call the FastAPI endpoints

**Use Case**: Connect AI agents to your own custom APIs and services.

---

## 🔧 Setup and Prerequisites

**Environment Variables Required:**
- `PROJECT_ENDPOINT`: Your Azure AI Project endpoint
- `MODEL_DEPLOYMENT_NAME`: Your deployed AI model name

**Additional Requirements:**
- FastAPI service running on `http://localhost:port`
- The `bank_transactions_api.py` file in the same directory

In [2]:
# Install required packages
# !pip install azure-ai-agents azure-identity semantic-kernel fastapi uvicorn requests 

import os
import json
import asyncio
import requests
import subprocess
import time
from azure.ai.agents.models import OpenApiTool, OpenApiAnonymousAuthDetails
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings
from semantic_kernel.contents import AuthorRole

print("✅ Packages imported successfully!")

✅ Packages imported successfully!


## 🚀 Step 1: Start the FastAPI Service

First, we need to start our bank transactions FastAPI service.

1. Open a terminal.
2. Navigate to the directory containing `bank_transactions_api.py`.
3. Activate your Python environment if necessary.
4. Run the FastAPI service using the command below.
5. Specify a port number (e.g., `8100`).
```bash
python bank_transactions_api.py 8100
```

In [ ]:
# Start the FastAPI service in the background
import subprocess
import time

print("🚀 Starting FastAPI bank transactions service...")

###############
## IMPORTANT ##
###############
# Start the FastAPI server in the background
# 1. Open a terminal, and make sure you are in the right directory
# 2. Activate your Python environment if needed
# 3. Run the following command with a specific port
#    Example: python bank_transactions_api.py 8000


# Test if the service is running
localhost = "http://localhost"
host="https://ca-agentfastapi.purpleplant-8215fdbe.swedencentral.azurecontainerapps.io"

http_port = 80  # Default port for FastAPI if using HTTP
https_port = 443 # Change to 443 for HTTPS 

if host == "":
    host = localhost
    port = http_port
else:
    port = https_port
    
print(f"🔍 Checking if FastAPI service is running on port {port}...")


try:
    full_host = f"{host}:{port}/"
    print(f"🔗 Full URL: {full_host}")
    response = requests.get(full_host)
    if response.status_code == 200:
        print("✅ FastAPI service is running!")
        print(f"Service info: {response.json()}")
    else:
        print(f"❌ Service responded with status: {response.status_code}")
except requests.exceptions.ConnectionError:
    print(f"❌ Could not connect to FastAPI service. Make sure it's running on port {port}.")
    print("💡 If you want to run it locally, you can start it manually with: python bank_transactions_api.py")

🚀 Starting FastAPI bank transactions service...
🔍 Checking if FastAPI service is running on port 443...
🔗 Full URL: https://ca-agentfastapi.purpleplant-8215fdbe.swedencentral.azurecontainerapps.io:443/
✅ FastAPI service is running!
Service info: {'service': 'Bank Transactions API', 'version': '1.0.0', 'endpoints': {'transactions': '/transactions', 'openapi': '/openapi.json', 'docs': '/docs'}}


## 🧪 Step 2: Test Individual API Endpoints

Let's also test the FastAPI endpoints directly to understand what data is available.

In [17]:
# Test the FastAPI endpoints directly
print("🧪 Testing FastAPI endpoints directly:")

# Test getting all transactions
try:
    response = requests.get(f"{host}:{port}/transactions")
    if response.status_code == 200:
        transactions = response.json()
        print(f"\n💳 Found {len(transactions)} transactions:")
        for txn in transactions:
            print(f"  {txn['date']}: {txn['description']} - ${txn['amount']:.2f} ({txn['category']})")
    else:
        print(f"❌ Failed to get transactions: {response.status_code}")
except Exception as e:
    print(f"❌ Error testing transactions endpoint: {e}")

# Test getting a specific transaction
try:
    response = requests.get(f"{host}:{port}/transactions/txn_001")
    if response.status_code == 200:
        transaction = response.json()
        print(f"\n🔍 Transaction txn_001 details:")
        print(f"  Date: {transaction['date']}")
        print(f"  Description: {transaction['description']}")
        print(f"  Amount: ${transaction['amount']:.2f}")
        print(f"  Balance: ${transaction['balance']:.2f}")
        print(f"  Category: {transaction['category']}")
    else:
        print(f"❌ Failed to get specific transaction: {response.status_code}")
except Exception as e:
    print(f"❌ Error testing specific transaction endpoint: {e}")


🧪 Testing FastAPI endpoints directly:

💳 Found 5 transactions:
  2024-12-01: Grocery Store Purchase - $-85.47 (Food & Dining)
  2024-12-02: Salary Deposit - $3500.00 (Income)
  2024-12-03: Gas Station - $-45.20 (Transportation)
  2024-12-04: Online Shopping - $-129.99 (Shopping)
  2024-12-05: Coffee Shop - $-5.75 (Food & Dining)

🔍 Transaction txn_001 details:
  Date: 2024-12-01
  Description: Grocery Store Purchase
  Amount: $-85.47
  Balance: $2914.53
  Category: Food & Dining


## 📋 Step 3: Fetch the OpenAPI Specification

Now let's fetch the OpenAPI spec from our running FastAPI service.

In [10]:
# Fetch the OpenAPI specification from the running service
try:    
    openapi_url = f"{host}:{port}/openapi.json"
    response = requests.get(openapi_url)
    
    if response.status_code == 200:
        bank_openapi_spec = response.json()
        print("📋 OpenAPI specification fetched successfully!")
        print(f"API Title: {bank_openapi_spec['info']['title']}")
        print(f"API Version: {bank_openapi_spec['info']['version']}")
        print(f"Available paths: {list(bank_openapi_spec['paths'].keys())}")
    else:
        print(f"❌ Failed to fetch OpenAPI spec: {response.status_code}")
        bank_openapi_spec = None
        
except requests.exceptions.ConnectionError:
    print("❌ Could not connect to FastAPI service to fetch OpenAPI spec")
    bank_openapi_spec = None

📋 OpenAPI specification fetched successfully!
API Title: Bank Transactions API
API Version: 1.0.0
Available paths: ['/', '/transactions', '/transactions/{transaction_id}']


### Adding the local FastAPI service to the Azure AI Project

In [ ]:
bank_openapi_spec["servers"] = [
    {
        "url": f"{host}:{port}",
    }
]

print(json.dumps(bank_openapi_spec, indent=2))

## 🤖 Step 4: Create Azure AI Agent with FastAPI Tools

Now we'll create an Azure AI Agent that can call our FastAPI endpoints.

In [ ]:
# Create Azure AI Agent with FastAPI OpenAPI tools
async def create_bank_agent():
    if bank_openapi_spec is None:
        print("❌ Cannot create agent without OpenAPI spec")
        return
    
    endpoint = os.environ["PROJECT_ENDPOINT"]
    model_deployment = os.environ["MODEL_DEPLOYMENT_NAME"]

    async with (
        DefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds, endpoint=endpoint) as client,
    ):
        print("🔗 Creating Azure AI Agent with FastAPI tools...")
        
        # Create OpenAPI tool for bank transactions
        auth = OpenApiAnonymousAuthDetails()
        
        bank_tool = OpenApiTool(
            name="bank_transactions",
            spec=bank_openapi_spec,
            description="Get bank transaction data from the local FastAPI service",
            auth=auth
        )
        
        # Create Azure AI agent with the FastAPI tool
        agent_definition = await client.agents.create_agent(
            model=model_deployment,
            name="bank_assistant",
            instructions="""You are a helpful bank assistant. Use the bank transactions API to help users understand their financial data. 
            Always provide clear explanations of transactions, categorize spending, and offer helpful insights about financial patterns.
            When showing transactions, format them nicely and explain what each field means.""",
            tools=bank_tool.definitions
        )
        
        # Wrap as AzureAIAgent
        bank_agent = AzureAIAgent(
            client=client,
            definition=agent_definition
        )
        
        print(f"🏦 Bank assistant agent created: {agent_definition.id}")
        
        # Test queries
        test_queries = [
            "Show me my latest bank transactions and tell me how much I spent on food.",
            "What was my largest expense recently and what category was it in?",
            "Can you give me a summary of my account balance and recent activity?"
        ]
        
        thread = None
        
        try:
            for i, query in enumerate(test_queries, 1):
                print(f"\n🧪 Test Query {i}:")
                print(f"👤 User: {query}")
                
                # Invoke the agent
                async for response in bank_agent.invoke(messages=query, thread=thread):
                    if response.role != AuthorRole.TOOL:
                        print(f"🏦 Bank Assistant: {response.content}")
                    thread = response.thread
                
                print("\n" + "="*50)
        
        finally:
            # Cleanup
            await client.agents.delete_agent(agent_definition.id)
            print("\n🧹 Bank agent cleaned up")

# Run the bank agent
await create_bank_agent()
print("\n✅ Bank assistant demonstration complete!")

🔗 Creating Azure AI Agent with FastAPI tools...
🏦 Bank assistant agent created: asst_hpHXoaeK7W3ai11IhCT53GQ1

🧪 Test Query 1:
👤 User: Show me my latest bank transactions and tell me how much I spent on food.
🏦 Bank Assistant: Here are your latest bank transactions:

1. Grocery Store Purchase
   - Date: 2024-12-01
   - Amount: -$85.47
   - Balance after transaction: $2,914.53
   - Category: Food & Dining

2. Salary Deposit
   - Date: 2024-12-02
   - Amount: +$3,500.00
   - Balance after transaction: $6,414.53
   - Category: Income

3. Gas Station
   - Date: 2024-12-03
   - Amount: -$45.20
   - Balance after transaction: $6,369.33
   - Category: Transportation

4. Online Shopping
   - Date: 2024-12-04
   - Amount: -$129.99
   - Balance after transaction: $6,239.34
   - Category: Shopping

5. Coffee Shop
   - Date: 2024-12-05
   - Amount: -$5.75
   - Balance after transaction: $6,233.59
   - Category: Food & Dining

Food Spending Summary:
You spent a total of $91.22 on food recently. Thi

ERROR:root:Error while closing connector: ClientConnectionError('Connection lost: SSL shutdown timed out')



✅ Bank assistant demonstration complete!


## 🛑 Step 5: Cleanup

Finally, let's stop the FastAPI service.

Please go to the shell, and kill the process running the FastAPI service. You can do this by pressing `Ctrl+C` in the terminal where the service is running.

---

## 🎯 Key Takeaways

✅ **FastAPI Integration:**
- FastAPI automatically generates OpenAPI specifications
- Azure AI Agents can consume any OpenAPI-compliant service
- Local development services work seamlessly with cloud agents

✅ **Dynamic API Discovery:**
- Fetch OpenAPI specs from running services
- No need to manually create API definitions
- Agents automatically understand available endpoints

✅ **Real-world Applications:**
- Connect agents to internal business APIs
- Integrate with existing microservices
- Create conversational interfaces for data services

### 💡 Best Practices:

🔒 **Security**: Use proper authentication for production APIs
📝 **Documentation**: Ensure OpenAPI specs have good descriptions
⚡ **Performance**: Consider rate limiting and caching
🧪 **Testing**: Always test both API endpoints and agent integration
🔄 **Error Handling**: Implement robust error handling in both API and agent

### 🚀 Next Steps:

- Add authentication to your FastAPI service
- Create more complex business logic APIs
- Combine multiple microservices with one agent
- Deploy both API and agents to production

**Happy API integrating!** 🎉